In [20]:
import json
import re
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from textblob import TextBlob
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict


In [2]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
with open('/content/dataset.txt', 'r', encoding='utf-8') as f:
    raw_json = f.read()

# The 'contradiction_spans' are stored as Python tuples (e.g., (0, 50)),
# which is not valid JSON. JSON requires arrays [0, 50].
# Also, Python booleans True/False need to be converted to JSON true/false.
# We need to preprocess the string to fix these.

corrected_json = re.sub(r'\((\d+,\s*\d+)\)', r'[\1]', raw_json)
corrected_json = corrected_json.replace('True', 'true').replace('False', 'false')

data = json.loads(corrected_json)
print("\nFirst review:")
data[0]


First review:


{'id': 1,
 'text': 'This laptop is incredibly fast. Boot time is under 10 seconds. However, I find myself waiting 5 minutes just to open Chrome. The performance is unmatched in this price range.',
 'has_contradiction': True,
 'contradiction_spans': [[0, 50], [51, 110]]}

In [4]:
t = data[0]['text'].lower()
t = re.sub(r'\s+', ' ', t).strip()
t

'this laptop is incredibly fast. boot time is under 10 seconds. however, i find myself waiting 5 minutes just to open chrome. the performance is unmatched in this price range.'

In [5]:
sentences = sent_tokenize(t)
sentences

['this laptop is incredibly fast.',
 'boot time is under 10 seconds.',
 'however, i find myself waiting 5 minutes just to open chrome.',
 'the performance is unmatched in this price range.']

In [7]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli')
labels = ['contradiction', 'neutral', 'entailment']


def find_sentences_for_span(span, text):

    start, end = span
    sentences = sent_tokenize(text)
    selected = []
    pos = 0
    for s in sentences:
        s_start = pos
        s_end = pos + len(s)
        if s_end >= start and s_start <= end:
            selected.append(s)
        pos += len(s) + 1
    return " ".join(selected).strip()


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:


final_nli_predictions = []

for item in data:
    if item['has_contradiction'] and len(item['contradiction_spans']) == 2:
        span1, span2 = item['contradiction_spans']


        sentence1 = find_sentences_for_span(span1, item['text'])
        sentence2 = find_sentences_for_span(span2, item['text'])


        encoded = tokenizer(sentence1, sentence2, return_tensors='pt')
        outputs = model(**encoded)
        probs = softmax(outputs.logits, dim=1)
        predicted_label = labels[torch.argmax(probs)]
        confidence = torch.max(probs).item()


        final_nli_predictions.append({
            'id': item['id'],
            'sentence1': sentence1,
            'sentence2': sentence2,
            'predicted_label': predicted_label,
            'confidence': confidence
        })


print("NLI predictions:")
for prediction in final_nli_predictions[:5]:
    print(prediction)


NLI predictions:
{'id': 1, 'sentence1': 'This laptop is incredibly fast. Boot time is under 10 seconds.', 'sentence2': 'Boot time is under 10 seconds. However, I find myself waiting 5 minutes just to open Chrome.', 'predicted_label': 'neutral', 'confidence': 0.9953163862228394}
{'id': 3, 'sentence1': "I've never had a phone this durable. Dropped it multiple times with no damage.", 'sentence2': 'Dropped it multiple times with no damage. The screen cracked on the first drop though.', 'predicted_label': 'neutral', 'confidence': 0.986533522605896}
{'id': 4, 'sentence1': 'Customer service was unhelpful and rude.', 'sentence2': "They resolved my issue within minutes and even gave me a discount. Worst support experience I've ever had.", 'predicted_label': 'contradiction', 'confidence': 0.9941118359565735}
{'id': 6, 'sentence1': 'Shipping was lightning fast - arrived in 2 days.', 'sentence2': 'Shipping was lightning fast - arrived in 2 days. The three-week wait was worth it though.', 'predicte

In [9]:

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')


def calculate_cosine_similarity(sentence1, sentence2):

    embeddings = model.encode([sentence1, sentence2])


    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    return similarity


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:


final_predictions = []

for item in data:
    if item['has_contradiction'] and len(item['contradiction_spans']) == 2:
        span1, span2 = item['contradiction_spans']


        sentence1 = item['text'][span1[0]:span1[1]].strip()
        sentence2 = item['text'][span2[0]:span2[1]].strip()


        similarity = calculate_cosine_similarity(sentence1, sentence2)


        threshold = 0.4
        is_contradiction = similarity < threshold

        final_predictions.append({
            'id': item['id'],
            'sentence1': sentence1,
            'sentence2': sentence2,
            'cosine_similarity': similarity,
            'is_contradiction': is_contradiction
        })


print("First 5 Predictions:")
for prediction in final_predictions[:5]:
    print(prediction)


First 5 Predictions:
{'id': 1, 'sentence1': 'This laptop is incredibly fast. Boot time is under', 'sentence2': '10 seconds. However, I find myself waiting 5 minutes just t', 'cosine_similarity': np.float32(0.33597565), 'is_contradiction': np.True_}
{'id': 3, 'sentence1': "I've never had a phone this durable. Dropped it multiple times with no", 'sentence2': 'damage. The screen cracked on the first drop', 'cosine_similarity': np.float32(0.53098255), 'is_contradiction': np.False_}
{'id': 4, 'sentence1': 'Customer service was unhelpful and rude.', 'sentence2': 'They resolved my issue within minutes and even gave me a discount. Wo', 'cosine_similarity': np.float32(0.2808818), 'is_contradiction': np.True_}
{'id': 6, 'sentence1': 'Shipping was lightning fast - arrived in 2 da', 'sentence2': 's. The three-week wait was worth it tho', 'cosine_similarity': np.float32(0.2590645), 'is_contradiction': np.True_}
{'id': 7, 'sentence1': 'This blender is whisper quiet. My baby sleeps right through', 's

In [11]:
def detect_manipulation(text):

    manipulation_keywords = [
        "hurry", "limited time", "only", "last chance", "buy now", "exclusive offer",
        "you deserve", "treat yourself", "must have", "don't miss out", "best deal"
    ]


    for keyword in manipulation_keywords:
        if keyword.lower() in text.lower():
            return True

    return False


final_manipulation_predictions = []

for item in data:

    is_manipulative = detect_manipulation(item['text'])


    final_manipulation_predictions.append({
        'id': item['id'],
        'text': item['text'],
        'is_manipulative': is_manipulative
    })


print("First 5 manipulation predictions:")
for prediction in final_manipulation_predictions[:5]:
    print(prediction)


First 5 manipulation predictions:
{'id': 1, 'text': 'This laptop is incredibly fast. Boot time is under 10 seconds. However, I find myself waiting 5 minutes just to open Chrome. The performance is unmatched in this price range.', 'is_manipulative': False}
{'id': 2, 'text': "The camera quality is stunning in daylight. Night mode works well too. I've taken beautiful photos at my daughter's evening recital. Great for any lighting condition.", 'is_manipulative': False}
{'id': 3, 'text': "I've never had a phone this durable. Dropped it multiple times with no damage. The screen cracked on the first drop though. Build quality is exceptional.", 'is_manipulative': False}
{'id': 4, 'text': "Customer service was unhelpful and rude. They resolved my issue within minutes and even gave me a discount. Worst support experience I've ever had.", 'is_manipulative': False}
{'id': 5, 'text': "The noise cancellation is mediocre at best. I can still hear my coworkers clearly. But honestly, for the price, you

In [21]:

urgency_keywords = ["only X left", "hurry", "limited time", "buy now", "act fast"]
scarcity_keywords = ["limited stock", "almost gone", "last chance", "only a few left"]
emotional_keywords = ["everyone is buying", "don't miss out", "you deserve this", "it's a must-have"]


def detect_manipulation(review_text):

    review_text = review_text.lower()

    urgency_found = any(keyword in review_text for keyword in urgency_keywords)

    scarcity_found = any(keyword in review_text for keyword in scarcity_keywords)

    emotional_found = any(keyword in review_text for keyword in emotional_keywords)

    sentiment = TextBlob(review_text).sentiment.polarity
    sentiment_score = "positive" if sentiment > 0 else "negative" if sentiment < 0 else "neutral"

    manipulation_score = 0


    if urgency_found or scarcity_found:
        manipulation_score += 1
    if emotional_found:
        manipulation_score += 1
    if sentiment_score == "positive":
        manipulation_score += 0.5


    is_manipulative = manipulation_score > 1.5

    return is_manipulative, manipulation_score, sentiment_score


review1 = "Only 2 left! I bought 3 just to be safe. You should too before they're gone!"
review2 = "I love this product! It changed my life."

print(detect_manipulation(review1))
print(detect_manipulation(review2))


(False, 0.5, 'positive')
(False, 0.5, 'positive')


In [22]:

def extract_style_features(text):

    words = text.split()
    sentences = re.split(r'[.!?]', text)
    sentences = [s for s in sentences if s.strip()]

    avg_sentence_len = len(words) / max(len(sentences), 1)
    avg_word_len = np.mean([len(w) for w in words]) if words else 0
    exclamations = text.count('!')
    questions = text.count('?')
    uppercase_ratio = sum(1 for c in text if c.isupper()) / max(len(text), 1)
    vocab_richness = len(set(words)) / max(len(words), 1)

    return [
        avg_sentence_len,
        avg_word_len,
        exclamations,
        questions,
        uppercase_ratio,
        vocab_richness
    ]


texts = [item['text'] for item in data]
review_ids = [item['id'] for item in data]

features = np.array([extract_style_features(t) for t in texts])

print("Feature matrix shape:", features.shape)


NUM_CLUSTERS = 3

kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
cluster_labels = kmeans.fit_predict(features)


clusters = defaultdict(list)

for idx, label in enumerate(cluster_labels):
    clusters[label].append({
        "id": review_ids[idx],
        "text": texts[idx]
    })


print("\n=== Stylistic Fingerprinting Results ===\n")

for cluster_id, reviews in clusters.items():
    print(f"\nCluster {cluster_id} ({len(reviews)} reviews):")

    for r in reviews:
        print(f"  - Review ID {r['id']}: {r['text'][:80]}...")

    if len(reviews) >= 3:
        print("  ⚠️  POSSIBLE SAME AUTHOR (Stylistic Match)")



Feature matrix shape: (8, 6)

=== Stylistic Fingerprinting Results ===


Cluster 2 (1 reviews):
  - Review ID 1: This laptop is incredibly fast. Boot time is under 10 seconds. However, I find m...

Cluster 0 (4 reviews):
  - Review ID 2: The camera quality is stunning in daylight. Night mode works well too. I've take...
  - Review ID 3: I've never had a phone this durable. Dropped it multiple times with no damage. T...
  - Review ID 6: Shipping was lightning fast - arrived in 2 days. The three-week wait was worth i...
  - Review ID 7: This blender is whisper quiet. My baby sleeps right through it. The noise is so ...
  ⚠️  POSSIBLE SAME AUTHOR (Stylistic Match)

Cluster 1 (3 reviews):
  - Review ID 4: Customer service was unhelpful and rude. They resolved my issue within minutes a...
  - Review ID 5: The noise cancellation is mediocre at best. I can still hear my coworkers clearl...
  - Review ID 8: Not the cheapest option, but definitely worth the premium price. The quality jus...
  ⚠

In [29]:

nli_dict = {pred['id']: pred for pred in final_nli_predictions}


final_labels = []

for item in data:
    review_id = item['id']


    if review_id in nli_dict:
        contradiction = nli_dict[review_id]['predicted_label'] == 'contradiction'
    else:
        contradiction = False


    manipulation = next((m['is_manipulative'] for m in final_manipulation_predictions if m['id'] == review_id), False)


    review_cluster_size = 1
    for cluster in clusters.values():
        if any(r['id'] == review_id for r in cluster):
            review_cluster_size = len(cluster)
            break
    same_author = review_cluster_size >= 3


    score = 0
    if contradiction:
        score += 1.0
    if manipulation:
        score += 0.7
    if same_author:
        score += 0.5


    if score >= 1.5:
        label = "Fake"
    elif score >= 0.7:
        label = "Suspicious"
    else:
        label = "Real"


    final_labels.append({
        "id": review_id,
        "contradiction": contradiction,
        "manipulation": manipulation,
        "same_author": same_author,
        "score": score,
        "final_label": label
    })

# Show results
print("=== Final Decision Engine Results ===")
for r in final_labels[:]:
    print(r)




=== Final Decision Engine Results ===
{'id': 1, 'contradiction': False, 'manipulation': False, 'same_author': False, 'score': 0, 'final_label': 'Real'}
{'id': 2, 'contradiction': False, 'manipulation': False, 'same_author': True, 'score': 0.5, 'final_label': 'Real'}
{'id': 3, 'contradiction': False, 'manipulation': False, 'same_author': True, 'score': 0.5, 'final_label': 'Real'}
{'id': 4, 'contradiction': True, 'manipulation': False, 'same_author': True, 'score': 1.5, 'final_label': 'Fake'}
{'id': 5, 'contradiction': False, 'manipulation': False, 'same_author': True, 'score': 0.5, 'final_label': 'Real'}
{'id': 6, 'contradiction': False, 'manipulation': False, 'same_author': True, 'score': 0.5, 'final_label': 'Real'}
{'id': 7, 'contradiction': True, 'manipulation': False, 'same_author': True, 'score': 1.5, 'final_label': 'Fake'}
{'id': 8, 'contradiction': False, 'manipulation': False, 'same_author': True, 'score': 0.5, 'final_label': 'Real'}


In [31]:
# ---------------------------
# FINAL OUTPUT RESULT
# ---------------------------

final_output = []

for idx, item in enumerate(data):
    review_id = item['id']

    # ----- A) Contradiction -----
    contradiction = False
    if idx < len(final_nli_predictions):
        contradiction = final_nli_predictions[idx]['predicted_label'] == 'contradiction'

    # ----- B) Manipulation -----
    manipulation = final_manipulation_predictions[idx]['is_manipulative']

    # ----- C) Stylometry / Same-author -----
    same_author = False
    for cluster_reviews in clusters.values():
        ids_in_cluster = [r['id'] for r in cluster_reviews]
        if review_id in ids_in_cluster and len(ids_in_cluster) > 2:  # arbitrary threshold
            same_author = True
            break

    # Combine evidence
    evidence = []
    if contradiction:
        evidence.append("Contradiction")
    if manipulation:
        evidence.append("Manipulation")
    if same_author:
        evidence.append("Fingerprint")

    # Simple confidence score (sum of signals / total possible)
    confidence = (int(contradiction) + int(manipulation) + int(same_author)) / 3

    # Decide final label
    is_deceptive = True if confidence >= 0.5 else False

    # Save final output
    final_output.append({
        'id': review_id,
        'deceptive': is_deceptive,
        'confidence': round(confidence, 2),
        'evidence': evidence
    })

# Print nicely
print("=== FINAL OUTPUT ===\n")
for out in final_output[:]:
    print(f"Review ID: {out['id']}")
    print(f"  Deceptive: {out['deceptive']}")
    print(f"  Confidence: {out['confidence']}")
    print(f"  Evidence: {', '.join(out['evidence']) if out['evidence'] else 'None'}")
    print("-" * 50)


=== FINAL OUTPUT ===

Review ID: 1
  Deceptive: False
  Confidence: 0.0
  Evidence: None
--------------------------------------------------
Review ID: 2
  Deceptive: False
  Confidence: 0.33
  Evidence: Fingerprint
--------------------------------------------------
Review ID: 3
  Deceptive: True
  Confidence: 0.67
  Evidence: Contradiction, Fingerprint
--------------------------------------------------
Review ID: 4
  Deceptive: False
  Confidence: 0.33
  Evidence: Fingerprint
--------------------------------------------------
Review ID: 5
  Deceptive: True
  Confidence: 0.67
  Evidence: Contradiction, Fingerprint
--------------------------------------------------
Review ID: 6
  Deceptive: False
  Confidence: 0.33
  Evidence: Fingerprint
--------------------------------------------------
Review ID: 7
  Deceptive: False
  Confidence: 0.33
  Evidence: Fingerprint
--------------------------------------------------
Review ID: 8
  Deceptive: False
  Confidence: 0.33
  Evidence: Fingerprint
-